# What

There is data used by the pytests to test the functionality of the engine and viewer.

I have previously somewhat randomly and manually created these data subsets to be used.

It would be better to have it in one place so that it is clear what data is used and needed.

## Modules

In [ ]:
import pandas as pd 

This is going to be ran assuming there is a output folder that has the expected files.

Rather than hook this up to the config file it is just a quick and simple hack to make it more transparent.

# Creating extracted dataset

In [ ]:
full_extracted_df = pd.read_pickle("../../output/extracted_reports.pkl")

full_extracted_df

In [ ]:
ids = [
    "2016_204",
    "2020_202",
    "2014_102",
    "2014_004",
    "2010_204",
    "2010_001",
    "2022_101",
    "2010_009",
    "2019_106",
]

# This is added as this is what was used in the previous extracted set which is used by alot of tests.
ids.extend(['2016_205',
 '2002_122',
 '2005_107',
 '2004_113',
 '2018_006',
 '2001_104',
 '2009_101',
 '2012_102'])

filtered_extracted_df = full_extracted_df.loc[ids]

filtered_extracted_df

In [ ]:
filtered_extracted_df.to_pickle("../../tests/data/output/extracted_reports.pkl")